In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import requests
import json
import pytz

In [180]:
def send_data(mydict,ts,devid,address):

    r1 = requests.get(
        url=address + "/api/device/" + devid + "/credentials",
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    devtoken = r1['credentialsId']
    
    my_json = json.dumps({'ts': ts, 'values': mydict})
    r = requests.post(url=address + "/api/v1/" + devtoken + "/telemetry",
                      data=my_json, headers={'Content-Type': 'application/json', 'Accept': '*/*',
                                             'X-Authorization': acc_token})
    return 
    


def check_latest(acc_token, address, start_time, end_time, agg):
    devid = 'e58dbff0-b5ab-11ec-ad6e-871d87776160'
    desc = 'wpump'
    r2 = requests.get(url=address + "/api/plugins/telemetry/DEVICE/" + devid + "/values/timeseries?keys=wpump&startTs=0&endTs="+end_time+"&agg=NONE&limit=1",
                headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    if (len(r2.keys())>0):
        print('wpump exists')
        df = pd.DataFrame(r2[desc])
        df.set_index('ts', inplace=True)
        df.columns = [str(desc)]
        df[desc] = df[desc].astype('float')
        
        # if water pump had opened
        if df[desc].iloc[0]>0:
            print('water pump is OPEN')
            if not agg.empty:
                if agg['class'].iloc[-1]==0: # pump was closed
                    ts = str(int(agg.index[-1].timestamp()) * 1000+86400000)
                    mydict={'wpump':str(agg['class'].iloc[-1])}
                    send_data(mydict,ts,devid,address)
            elif agg.empty:
                if (int(end_time) - df[desc].index[0])>300000: # more than 5 minutes have passed from pump-ON
                    #ALARM
                    print('alarm')
        else:
            print('water pump is CLOSED')
        
        # if water pump was closed
    else:
        print('wpump does not exist')
        if not agg.empty: 
            ts = str(int(agg.index[-1].timestamp()) * 1000)
            mydict={'wpump':str(agg['class'].iloc[-1])}
            send_data(mydict,ts,devid,address)
            
        
                
                

In [31]:
def read_data(acc_token, address, start_time, end_time):
    devid = 'e58dbff0-b5ab-11ec-ad6e-871d87776160'
    descriptors='pwrB,rpwrB'
    df = pd.DataFrame([])
    
    if int(end_time)-int(start_time)>(30*86400000):
        offset = 30*86400000
    else:
        offset = 86400000
    svec = np.arange(int(start_time), int(end_time), offset) # 1 day
    for st in svec:
        en = st + offset - 1

        if int(end_time) - en <= 0: en = int(end_time)
        tmp = pd.DataFrame([])

        r2 = requests.get(url=address + "/api/plugins/telemetry/DEVICE/" + devid + "/values/timeseries?keys="+descriptors+"&startTs=" + start_time + "&endTs=" + end_time + "&agg=NONE&limit=1000000",
            headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
        if ((len(r2.keys())>0) & (len(descriptors)>0)):


            # read all descriptors at once
            for desc in r2.keys():
                try:
                    df1 = pd.DataFrame(r2[desc])
                    df1.set_index('ts', inplace=True)
                    df1.columns = [str(desc)]
                    tmp = pd.concat([tmp,df1], axis = 1)
                except:
                    continue


            if not tmp.empty:

                tmp.reset_index(drop=False, inplace=True)
                tmp['ts'] = pd.to_datetime(tmp['ts'], unit='ms')
                tmp['ts'] = tmp['ts'].dt.tz_localize('utc').dt.tz_convert('Europe/Athens')
                # Set timestamp as index, convert all columns to float
                tmp = tmp.sort_values(by=['ts'])
                tmp.reset_index(drop=True, inplace=True)
                tmp.set_index('ts',inplace = True, drop = True)
                df = pd.concat([df, tmp])

                      
    if not df.empty:
        for col in df.columns:
            df[col] = df[col].astype('float')
        #df = df.add_suffix('_'+str(incr))
                
        df = df.resample('250ms').max()
    return df


In [187]:
mdlON = pickle.load(open('mdlON', 'rb'))
mdlOFF = pickle.load(open('mdlOFF', 'rb'))
thresON = 100
thresOFF = -100
thresOFF2 = -300

address = 'http://51.103.232.223:8080'

r = requests.post(address + "/api/auth/login",json={'username': 'tenant@thingsboard.org', 'password': 'tenant'}).json()
acc_token = 'Bearer' + ' ' + r['token']

start_time = '1650463200000'
end_time = '1650463800000'

In [188]:
df = read_data(acc_token, address, start_time, end_time)
for col in df.columns:
    df[col] = df[col].fillna(method='ffill')
df['eventON']=0
df['eventOFF']=0
df['avg1'] = df['pwrB'].rolling(4).mean()
df['avg2'] = df['pwrB'].shift(4).rolling(4).mean()
df['avgR'] = df['rpwrB'].shift(4).rolling(4).mean()

df['avgP_event'] = df['pwrB'].rolling(4).mean().shift(-4)-df['avg2']
df['stdP_event'] = df['pwrB'].rolling(12).std().shift(-4)
df['avgR_event'] = np.abs(df['rpwrB'].rolling(4).mean().shift(-4)-df['avgR'])
df['stdR_event'] = df['rpwrB'].rolling(12).std().shift(-4)


df.loc[(df['avg1']-df['avg2'])>thresON,'eventON']=1
df.loc[((df['avg1']-df['avg2'])<thresOFF) & ((df['avg1']-df['avg2'])>thresOFF2),'eventOFF']=1
df.drop(['avg1','avg2','avgR'],axis=1,inplace=True)

In [192]:

# run model ON
tmp1 = df.copy()
tmp1.loc[(tmp1['eventON']==1)&(tmp1['eventON'].shift()==1),'eventON']=0
tmp1 = tmp1.loc[tmp1['eventON']==1]

if not tmp1.empty:
    tmp1 = tmp1[['avgP_event','stdP_event','avgR_event','stdR_event']]

    labelsON = mdlON.predict(tmp1)
    tmp1['class']=labelsON
    tmp1 = tmp1.loc[tmp1['class']==1]


# run model OFF
tmp2 = df.copy()
tmp2.loc[(tmp2['eventOFF']==1)&(tmp2['eventOFF'].shift()==1),'eventOFF']=0
tmp2 = tmp2.loc[tmp2['eventOFF']==1]

if not tmp2.empty:
    tmp2 = tmp2[['avgP_event','stdP_event','avgR_event','stdR_event']]

    labelsOFF = mdlOFF.predict(tmp2)
    tmp2['class']=labelsOFF
    tmp2 = tmp2.loc[tmp2['class']==1]
    tmp2['class']=0

if (not tmp1.empty)&(not tmp2.empty):
    # concatenate
    agg = pd.concat([tmp1,tmp2])
    agg.sort_index(inplace=True)
elif not tmp2.empty:
    agg = tmp2.copy()
elif not tmp1.empty:
    agg = tmp2.copy()
else:
    agg = pd.DataFrame([])
    

# run check
check_latest(acc_token, address, start_time, end_time, agg)